# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1.  Изучение общей информации

In [1]:
import pandas as pd #импортировали библиотеку pandas
data=pd.read_csv('/datasets/data.csv') #прочитали файл и сохранили в переменной data
display(data.head(30)) # выводим на экран первые 30 строк таблицы
data.info() #получаем сводную информацию о таблице

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Таблица имеет 21525 строки и 12 столбцов. На первый взгляд видно, что в столбце days_employed есть отрицательные значения и цифры слишком большие для дней стажа(возможно часы?), нужно проверить корректность. Столбцы days_employed и total_income содержат вещественные цифры, лучше их преобразовать в целые и в этих столбцах есть пропуски, судя по количеству строк из сводной информации. Также в столбце education используются прописные и строчные буквы,а в столбце purpose нет единого написания целей. Данные нужны обработать и привести в надлежащий вид для корректного анализа.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [15]:
display(data.describe()) # проверяем описательную статистику по всем столбцам
for income in data['income_type'].unique(): # цикл для столбцов dob_years , который заполнит пропуски средним значением этого столбца в зависимости от income_type
    mean_years = data.loc[data['income_type'] == income, 'dob_years'].mean().astype('int')
    print(income,mean_years)
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == income), 'dob_years'] = mean_years
print(data['dob_years'].min()) # вывела минимальный возраст для проверки, что это не 0

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,21454.000000,2.145400e+04
mean,0.480563,4592.810245,43.473758,0.817097,0.973898,0.081150,1.653201e+05
std,0.756069,5348.248346,12.225661,0.548674,1.421567,0.273072,9.818730e+04
min,0.000000,24.000000,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1023.000000,33.000000,1.000000,0.000000,0.000000,1.076239e+05
50%,0.000000,1996.000000,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,5320.000000,53.000000,1.000000,1.000000,0.000000,1.958209e+05
max,5.000000,18388.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


сотрудник 40
пенсионер 59
компаньон 39
госслужащий 40
безработный 38
предприниматель 42
студент 22
в декрете 39
19


В столбце dob_years были нулевые значения, предполагаем, что это пропуски. Заполнили их средним значением в зависимости от типа занятости. Теперь минимальный возраст равен 19 годам

In [3]:
print('Пропуски до:', data['days_employed'].isna().sum()) # выводим на экран количество пропусков в столбце days_employed
print('Пропуски до:', data['total_income'].isna().sum()) #выводим на экран количество пропусков в столбце total_income
data['days_employed'] = data['days_employed'].abs() # беру по модулю данные в столбце days_employed т.к. есть отрицательные значения
data.loc[data['days_employed'] >300000, 'days_employed'] = data.loc[data['days_employed'] >300000, 'days_employed'] / 24 #перевожу значения > 300000 в столбце days_employed в часы


for income in data['income_type'].unique(): # цикл для столбцов days_employed и total_income , который заполнит пропуски медианными значениями этих столбцов в зависимости от income_type
    median_days = data.loc[data['income_type'] == income, 'days_employed'].median()
    print(income,median_days)
    data.loc[(data['days_employed'].isna()) & (data['income_type'] == income), 'days_employed'] = median_days

    median_total = data.loc[data['income_type'] == income, 'total_income'].median()
    print(income,median_total)
    data.loc[(data['total_income'].isna()) & (data['income_type'] == income), 'total_income'] = median_total
print('Пропуски после:', data['days_employed'].isna().sum()) # выводим на экран количество пропусков в столбце days_employed после обработки
print('Пропуски после:', data['total_income'].isna().sum()) ##выводим на экран количество пропусков в столбце total_income после обработки


    



    




Пропуски до: 2174
Пропуски до: 2174
сотрудник 1574.2028211070851
сотрудник 142594.39684740017
пенсионер 15217.221094405466
пенсионер 118514.48641164352
компаньон 1547.3822226779334
компаньон 172357.95096577113
госслужащий 2689.3683533043886
госслужащий 150447.9352830068
безработный 15267.235531008522
безработный 131339.7516762103
предприниматель 520.8480834953765
предприниматель 499163.1449470857
студент 578.7515535382181
студент 98201.62531401133
в декрете 3296.7599620220594
в декрете 53829.13072905995
Пропуски после: 0
Пропуски после: 0


**Вывод**

В столбцах days_employed и total_income действительно были пропуски, причем одинаковое количество. Данные из столбца total_income нам нужны для анализа и удалять мы их не можем. Пропуски могли возникнуть из-за сбора информации из разных источников, например, соединили две таблицы и какие-то данные были утеряны. В столбце days_employed мы видим отрицательные значения и очень большие положительные значения для дней.Это могло возникнуть из-за разных единиц измерения данных в разных источниках. Где-то считали стаж в часах, где-то в днях. В столбце days_employed взяты значения по модулю, теперь все значения положительные, а значения больше 300000 переведены в часы. Пропуски  в столбцах заполнила медианными значениями в зависимости от типа занятости (income_type). Теперь пропусков в исходном датафрейме нет.

### Замена типа данных

In [4]:

data['days_employed']=data['days_employed'].astype('int') # смена типа данных с вещественного на целый в столбце days_employed
data.head(10) # для проверки выводим первые 10 строк датафрейма

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В столбцах days_employed и total_income вещественные числа. Дни стажа считаются целыми числами, поэтому в столбце days_employed поменяла тип данных на целые числа. Столбец с ежемесячным доходом можно оставить с вещественными числами.

<div class="alert alert-success">
<b>Комментарий ревьюера✅: </b> Согласна. Но в целом доход тоже можно привести к целочисленным, так ка точность до секунд и копеек была бы излишней.   </div>

### Обработка дубликатов

In [5]:
display(data['education'].unique()) # ищем уникальные значения в столбце education
data['education'] = data['education'].str.lower() # приводим к единому написанию, применив ко всему столбцу education сторчоное написание букв
print(data['education'].unique()) # проверяем
data = data.drop_duplicates().reset_index(drop = True) #удаляем возможные дубликаты строк
data.info() # проверяем через просмотр сводной информации о датафрейме

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

В столбце education привели все к единому написанию. Также в таблице не надо исключать дубликаты, которые могли образоваться из-за слияние нескольких файлов при создании данного датафрейма или просто при вводе данных, человеческий фактор. Поэтому удалили дубликаты строк с обновлением индексации. После удаления количество строк уменьшилось. 

<div class="alert alert-success">
<b>Комментарий ревьюера✅: </b> Все верно. Сначала нужно было разобраться со скрытыми дублями, а потом переходить к явным.   </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера ⚠ : </b> Однако перед удалением дубликатов всегда нужно выводить их количество, чтобы не удалить слишком много.   </div>

### Лемматизация

In [16]:
from pymystem3 import Mystem #импортируем библиотеку pymystem3
m = Mystem()
from collections import Counter #вызываем специальный контейнер Counter из модуля collections для посчета упоминаний слов в столбце purpose
text = data['purpose'] # присвоение переменной text  значений в столбце purpose  
text = ' '.join(text) # присоединяем пробел между уникальными значениями в столбце purpose
lemmas = m.lemmatize(text) #лемматизируем и сохраняем в переменной lemmas

print(lemmas) #выводим на экран результат лемматизации
print(Counter(lemmas)) # выводим на экран количество уникальных значений
def category(row): # создаем функцию для создания категорий для столбца purpose, возвращает категорию по результату лемматизации столбца purpose
    purpose = row['purpose']
    purpose_common = m.lemmatize(purpose)
    if ('недвижимость' in purpose_common or 'жилье' in purpose_common):
        return 'недвижимость'
    elif 'автомобиль' in purpose_common:
        return 'автомобиль'
    elif 'образование' in purpose_common:
        return 'образование'
    elif 'свадьба' in purpose_common:
        return 'свадьба'
    else:
        return 'цель не определена'
data['purpose_common'] = data.apply(category, axis = 1) # создаем столбец purpose_common для категорий
data.head(10) # выводим на экран первые 10 строк датафрейма

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'покупка', ' ', 'жилье', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'сыграть', ' ', 'свадьба', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'недвижимость', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'сыграть', ' ', 'свадьба', ' ', 'опера

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_common,have_children,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,дети есть,средний уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,дети есть,средний уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,детей нет,средний уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,дети есть,средний уровень дохода
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,детей нет,средний уровень дохода
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость,детей нет,средний уровень дохода
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость,детей нет,средний уровень дохода
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование,детей нет,средний уровень дохода
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба,дети есть,средний уровень дохода
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость,детей нет,средний уровень дохода


**Вывод**

После лемматизации можно выделить несколько категорий столбца purpose: недвижимость, автомобиль, образование, свадьба. Данные категории выделены как наиболее встречающиеся и обобщающие слова в столбце purpose. Для дальнейшего анализа будем использовать созданный столбец purpose_common с выделенными категориями после лемматизации.

### Категоризация данных

In [7]:
display(data['children'].drop_duplicates().reset_index(drop = True))#категоризируем данные в столбце children,видим значения -1 и 20
display(data['children'].value_counts()) #смотрим количество этих значений, возможно данные некорректные,-1 ребенок не может быть и 20 детей в таком количестве тоже вряд ли
data['children'] = data['children'].abs() # берем значения по модулю
data.loc[data['children'] == 20, 'children'] = 2 # предполагаю, что вместо 20 должно быть 2, присваиваем вместо 20 - 2
display(data['children'].drop_duplicates().reset_index(drop = True))# снова категоризируем данные в столбце children
def have_children(child): # создаем функцию, которая возвращает результат о наличии детей в зависимости от значения
    if child == 0:
        return 'детей нет'
    else: return 'дети есть'

# добавляем новый столбец по категориям (применяя функцию)
data['have_children'] = data['children'].apply(have_children) #добавляем столбец have_children с информации о наличии детей
display(data.head(20)) #выводим на экран первые 20 строк датафрейма
    

0     1
1     0
2     3
3     2
4    -1
5     4
6    20
7     5
Name: children, dtype: int64

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

0    1
1    0
2    3
3    2
4    4
5    5
Name: children, dtype: int64

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_common,have_children
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,дети есть
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,дети есть
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,детей нет
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,дети есть
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,детей нет
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость,детей нет
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость,детей нет
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование,детей нет
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба,дети есть
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость,детей нет


In [8]:
display(data['total_income'].max()) #столбец total_income лучше разбить по уровнб дохода
display(data['total_income'].min()) #посмотрим максимальное, минимальное и медианное значения столбца для выделения групп
display(data['total_income'].median())
def income_level(income): # создаем функцию, которая возвращает уровень дохода в зависимости от значения
    if income < 40000:
        return 'низкий уровень дохода'
    elif income < 300000:
        return 'средний уровень дохода'
    elif income < 1000000:
        return 'высокий уровень дохода'
    else:
        return 'очень высокий уровень дохода'
# добавляем новый столбец по категориям (применяя функцию)
data['income_level'] = data['total_income'].apply(income_level) #добавляем столбец income_level со значениями уровня дохода
display(data.head(20)) #выводим на экран первые 20 строк датафрейма

2265604.028722744

20667.26379327158

142594.39684740017

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_common,have_children,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,недвижимость,дети есть,средний уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,автомобиль,дети есть,средний уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,недвижимость,детей нет,средний уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,образование,дети есть,средний уровень дохода
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,свадьба,детей нет,средний уровень дохода
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,недвижимость,детей нет,средний уровень дохода
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,недвижимость,детей нет,средний уровень дохода
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,образование,детей нет,средний уровень дохода
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,свадьба,дети есть,средний уровень дохода
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,недвижимость,детей нет,средний уровень дохода


In [17]:
display(data['income_type'].drop_duplicates().reset_index(drop = True))#категоризируем данные в столбце income_type
display(data['debt'].drop_duplicates().reset_index(drop = True)) #категоризируем данные в столбце debt
display(data['purpose_common'].drop_duplicates().reset_index(drop = True)) #категоризируем данные в столбце purpose_common
display(data['family_status'].drop_duplicates().reset_index(drop = True))#категоризируем данные в столбце family_status


0          сотрудник
1          пенсионер
2          компаньон
3        госслужащий
4        безработный
5    предприниматель
6            студент
7          в декрете
Name: income_type, dtype: object

0    0
1    1
Name: debt, dtype: int64

0    недвижимость
1      автомобиль
2     образование
3         свадьба
Name: purpose_common, dtype: object

0          женат / замужем
1         гражданский брак
2           вдовец / вдова
3                в разводе
4    Не женат / не замужем
Name: family_status, dtype: object

**Вывод**

После категоризации мы видим, что все столбцы для анализа с корректными данными. По столбцу children мы сделали фильтрацию по наличю детей. Дале будем использовать столбец have_children.По столбцу total_income сделали фильтрацию по уровню доходу. Для анализа будет использовать столбец income_level. По остальным столбцам проверили, что информация и значения корректные для анализа.

## Шаг 3. Расчеты

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
data_pivot = data.pivot_table(index = ['have_children'], values = 'debt', aggfunc = ['sum','count','mean']) #создаем сводную таблицу с посчетом суммы,
data_pivot.columns = ['debt','total','%']                                                              # количества и среднего значения столбца debt с группировкой по столбцу children
data_pivot['%'] = data_pivot['%']*100
data_pivot['%'] = data_pivot['%'].round(2)
data_pivot = data_pivot.sort_values(by = '%',ascending = False) #сортируем значения столбца % по убыванию
data_pivot

,debt,total,%
have_children,,,
дети есть,678,7363,9.21
детей нет,1063,14091,7.54


**Вывод**

Исходя из сводной таблицы можно отметить, что вероятность возврата кредита в срок выше у людей с  детьми, чем без. Значит зависимость наблюдается.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
data_pivot = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum','count','mean'])#создаем сводную таблицу с посчетом суммы,
data_pivot.columns = ['debt','total','%']   # количества и среднего значения столбца debt с группировкой по столбцу family_status
data_pivot['%'] = data_pivot['%']*100
data_pivot['%'] = data_pivot['%'].round(2)
data_pivot = data_pivot.sort_values(by = '%',ascending = False)#сортируем значения столбца % по убыванию
data_pivot

,debt,total,%
family_status,,,
Не женат / не замужем,274,2810,9.75
гражданский брак,388,4151,9.35
женат / замужем,931,12339,7.55
в разводе,85,1195,7.11
вдовец / вдова,63,959,6.57


**Вывод**

Самый большой процент возврата кредита в срок у не женатых/не замужних и тех, кто состоит в гражданском браке. Самый низкий - у вдовцов/вдов. Завимость прослеживается, но небольшая.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
data_pivot = data.pivot_table(index = ['income_level'], values = 'debt', aggfunc = ['sum','count','mean'])#создаем сводную таблицу с посчетом суммы,
data_pivot.columns = ['debt','total','%']                                                                 # количества и среднего значения столбца debt с группировкой по столбцу income_level
data_pivot['%'] = data_pivot['%']*100
data_pivot['%'] = data_pivot['%'].round(2)
data_pivot = data_pivot.sort_values(by = '%',ascending = False)#сортируем значения столбца % по убыванию
data_pivot

,debt,total,%
income_level,,,
средний уровень дохода,1625,19846,8.19
низкий уровень дохода,10,125,8.00
очень высокий уровень дохода,2,25,8.00
высокий уровень дохода,104,1458,7.13


**Вывод**

Зависимости между уровнем дохода и возвратом кредита в срок я не отмечаю. Все примерно на одном уровне.

- Как разные цели кредита влияют на его возврат в срок?

In [13]:
data_pivot = data.pivot_table(index = ['purpose_common'], values = 'debt', aggfunc = ['sum','count','mean'])#создаем сводную таблицу с посчетом суммы,
data_pivot.columns = ['debt','total','%']                                                                   # количества и среднего значения столбца debt с группировкой по столбцу purpose_commom
data_pivot['%'] = data_pivot['%']*100
data_pivot['%'] = data_pivot['%'].round(2)
data_pivot = data_pivot.sort_values(by = '%',ascending = False)#сортируем значения столбца % по убыванию
data_pivot

,debt,total,%
purpose_common,,,
автомобиль,403,4306,9.36
образование,370,4013,9.22
свадьба,186,2324,8.00
недвижимость,782,10811,7.23


**Вывод**

Самый больщой процент возврата кредита в срок у людей, которые покупают автомобили, самый низкий - при покупке недвижимости. Зависимость между вовзратом кредита в срок и целей кредита есть, но тоже не существенная.

## Шаг 4. Общий вывод

Ярко выраженной зависимости семейного положения, количество детей клиента, уровня дохода, целей кредита на факт погашения кредита в срок не найдено, так как разница процента вероятности возврата не существенная. На основе полученных данных мы можем составить потенциального идеального клиента кредитного отдела банка: не женат/не замужем с любым уровнем дохода, имеет детей, берет кредит на покупку автомобиля.